# Using The Model For Prediction

### Step-1: Loading the Model Created using model_training.ipynb

In [8]:
from keras.models import load_model
model=load_model("mnist_model1.h5")

### Step-2: Loading the Testing and Training data for prediction

In [12]:
import gzip
import struct 
import numpy as np
from keras.utils import np_utils
def read_idx(filename):
    with gzip.open(filename, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.frombuffer(f.read(), dtype=np.uint8).reshape(shape)
        #np.fromstring(f.read(), dtype=np.uint8).reshape(shape)
        
x_test = read_idx("C:/Users/Rishabh/summertraining2021/Tasks/MLProton Internship Assignment//data/t10k-images-idx3-ubyte.gz")
y_test = read_idx("C:/Users/Rishabh/summertraining2021/Tasks/MLProton Internship Assignment/data/t10k-labels-idx1-ubyte.gz")

# Getting our data in the right 'shape' needed for Keras
# We need to add a 4th dimenion to our date thereby changing our
# Our original image shape of (60000,28,28) to (60000,28,28,1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)


# change our image type to float32 data type
x_test = x_test.astype('float32')

# Normalize our data by changing the range from (0 to 255) to (0 to 1)
x_test /= 255

# Now we one hot encode outputs
y_test = np_utils.to_categorical(y_test)

# Number of classed for classification
num_classes = y_test.shape[1]

### Step-3: Creating a Function to check for which class does the Prediction Belongs

In [15]:
import cv2
import numpy as np

#Refer readme.md file to know more about this.
def getLabel(input_class):
    number = int(input_class)
    if number == 0:
        return "T-shirt/top "
    if number == 1:
        return "Trouser"
    if number == 2:
        return "Pullover"
    if number == 3:
        return "Dress"
    if number == 4:
        return "Coat"
    if number == 5:
        return "Sandal"
    if number == 6:
        return "Shirt"
    if number == 7:
        return "Sneaker"
    if number == 8:
        return "Bag"
    if number == 9:
        return "Ankle boot"


### Step-4: Creating a Function to Compare predicted and Actual Image

In [16]:
def draw_test(name, pred, actual, input_im):
    BLACK = [0,0,0]
    res = getLabel(pred)
    actual = getLabel(actual)   
    new_img_expanded = cv2.copyMakeBorder(input_im, 0, 0, 0, 4*imageL.shape[0] ,cv2.BORDER_CONSTANT,value=BLACK)
    new_img_expanded = cv2.cvtColor(new_img_expanded, cv2.COLOR_GRAY2BGR)
    cv2.putText(new_img_expanded, "Predicted - " + str(res), (152, 70) , cv2.FONT_HERSHEY_COMPLEX_SMALL,1, (0,255,0), 1)
    cv2.putText(new_img_expanded, "   Actual - " + str(actual), (152, 90) , cv2.FONT_HERSHEY_COMPLEX_SMALL,1, (0,0,255), 1)
    cv2.imshow(name, new_img_expanded)


### Step-5:Testing on 10 Random Images using Above created functions

In [22]:

for i in range(0,10):
    rand = np.random.randint(0,len(x_test))
    input_im = x_test[rand]
    actual = y_test[rand].argmax(axis=0)
    imageL = cv2.resize(input_im, None, fx=4, fy=4, interpolation = cv2.INTER_CUBIC)
    #we trained the data in 4d shape so we also need the 4d shape to predict the data 
    input_im = input_im.reshape(1,28,28,1) 
    
    ## Get Prediction
    res = str(np.argmax(model.predict(input_im, 1, verbose = 0), axis=-1)[0])
    #res = str(model.predict_classes(input_im, 1, verbose = 0)[0])

    draw_test("Prediction", res, actual, imageL) 
    cv2.waitKey(0)

cv2.destroyAllWindows()